In [72]:
#Importamos todas las librerías que vamos usar y el archivo de keys para acceder
import requests
import json
from IPython.display import JSON
from keys import *

In [73]:
#Ponemos la url de la api de los vuelos con el código de nuestra ciudad y país en este caso es YMML que es para melbourne 

url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/YMML/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

#Ponemos nuestros datos proporcionados por la api y la cuenta que hicimos en la página
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': "77cd5e6190mshbabec6d3965dd94p1667efjsn21302f39a238"
    }

response = requests.request("GET", url, headers=headers, params=querystring)


In [74]:
#generamos el json con todos los datos que estamos extrayendo

if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [75]:
#Llenamos y formamos el json con los datos que queremos para flights arrivals

flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']['utc']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']['terminal']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']



In [79]:
#importamos pandas
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

#hacemos la extracción de los datos y los ponemos en tabla  de los primeros 5 datos con el flight_arrivals.head

#def flight_extraction(flight):
#    flights_df = pd.json_normalize({
 #       'Scheduled_arrival_time': flight['arrival']['scheduledTime']['utc'],
  #      'flight_number': flight['number'],
   #     'from' : flight['departure']['airport']['name'],
    #    'airline' : flight['airline']['name'],
    #})
    #return(flights_df)


def flight_extraction(flight):
    flights_df = pd.json_normalize({
        'scheduled_arrival_time': flight['arrival']['scheduledTime']['utc'],  
        'flight_number': flight['number'],
        'from_airport': flight['departure']['airport']['name'],
        'airline': flight['airline']['name'],
        #'icao_code': f"NULL"
    })
    return(flights_df)


flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']],
    ignore_index=True
)

flight_arrivals.head()

,scheduled_arrival_time,flight_number,from_airport,airline
0,2025-08-31 21:55Z,VA 204,Adelaide,Virgin Australia
1,2025-08-31 22:25Z,VA 726,Gold,Virgin Australia
2,2025-08-31 22:10Z,TG 4844,Wellington,Thai International
3,2025-08-31 22:10Z,NZ 253,Wellington,Air New Zealand
4,2025-08-31 22:10Z,VA 7153,Wellington,Virgin Australia


In [80]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "al0294rR651%J"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [81]:
# send the weather data to the database
flight_arrivals.to_sql('flight_arrival', if_exists = 'append', con = con, index=False)

319